In [ ]:
!pip install reportlab

In [ ]:
!pip install fuzz

In [ ]:
!pip install nltk

In [ ]:
!pip install fuzzywuzzy

In [ ]:
import random
import nltk
import time
from tqdm import tqdm
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from fuzzywuzzy import fuzz
from nltk.corpus import words as nltk_words
from nltk.tokenize import word_tokenize


In [ ]:
# ---- Constants ----
SIMILARITY_THRESHOLD = 50
DYSLEXIA_SCORE_THRESHOLD = 3.5

# Simple mapping for questionnaire answers to numeric scores
ANSWER_SCORE_MAP = {
    "never": 0,
    "lessOften": 1,
    "sometimes": 2,
    "moreOften": 3
}

# Ensure necessary NLTK resources are available
def ensure_nltk_resources():
    try:
        nltk.data.find('tokenizers/punkt')
    except LookupError:
        nltk.download('punkt')
    try:
        nltk.data.find('corpora/words')
    except LookupError:
        nltk.download('words')

ensure_nltk_resources()

# Create a set of English words from NLTK
english_vocab = set(w.lower() for w in nltk_words.words())

# Dyslexic confusions (kept from your original)
DYSLEXIC_LETTER_CONFUSIONS = [
    ('b', 'd'), ('p', 'q'), ('m', 'w'), ('n', 'u'), ('n', 'r'),
    ('i', 'j'), ('a', 'e'), ('s', 'z'), ('f', 't'), ('c', 'k'),
    ('g', 'q'), ('h', 'n'), ('v', 'w'), ('b', 'p'), ('c', 's'),
    ('d', 't'), ('o', 'e'), ('a', 'o'), ('u', 'v'), ('m', 'n')
]

DYSLEXIC_WORD_CONFUSIONS = [
    ('was', 'saw'), ('there', 'their'), ('here', 'hear'),
    ('you', 'your'), ('where', 'wear'), ('to', 'too', 'two'),
    ('its', "it's"), ('right', 'write'), ('flower', 'flour'),
    ('buy', 'by', 'bye'), ('no', 'know'), ('for', 'four'), ('sun', 'son'),
    ('allowed', 'aloud'), ('hour', 'our'), ('blew', 'blue'), ('sew', 'sow'),
    ('be', 'bee'), ('one', 'won')
]


In [ ]:
# ---- Text utilities ----
def tokenize_and_clean_text(text):
    words = word_tokenize(text.lower())
    return [word for word in words if word.isalnum()]

def is_exact_match(user_text, random_sentence):
    return user_text.lower().strip() == random_sentence.lower().strip()

def are_strings_similar(str1, str2, threshold):
    return fuzz.ratio(str1.lower(), str2.lower()) >= threshold

# ---- Dyslexia scoring (kept and slightly simplified) ----
def calculate_word_dyslexia_score(word, random_sentence):
    word_tokens = tokenize_and_clean_text(word)
    sentence_tokens = tokenize_and_clean_text(random_sentence)
    dyslexia_score = 0.0

    # defensive checks
    if not word_tokens or not sentence_tokens:
        return 0.0

    # if full match reduce score (encourage correct copying)
    if " ".join(word_tokens) == " ".join(sentence_tokens):
        return 0.0

    for word_token in word_tokens:
        token_dyslexia_score = 0.0

        if word_token not in english_vocab:
            token_dyslexia_score += 3.0

        # letter-pair confusions (if both letters appear in token)
        for confusion in DYSLEXIC_LETTER_CONFUSIONS:
            if confusion[0] in word_token and confusion[1] in word_token:
                token_dyslexia_score += 1.2  # smaller increment

        # word-level confusion (if token contains confusion words)
        for confusion in DYSLEXIC_WORD_CONFUSIONS:
            # if token equals or contains any confusion forms
            if any(conf_word == word_token for conf_word in confusion):
                token_dyslexia_score += 2.0

        # transposition check vs first token of sentence
        ref = sentence_tokens[0]
        if len(word_token) == len(ref):
            transpositions = [(word_token[:i] + word_token[i + 1] + word_token[i] + word_token[i + 2:]) for i in range(len(word_token) - 1)]
            if ref in transpositions:
                token_dyslexia_score += 1.8

        # reversed word
        if word_token[::-1] == ref:
            token_dyslexia_score += 1.8

        dyslexia_score += token_dyslexia_score

        if token_dyslexia_score == 0:
            dyslexia_score -= 0.3

    return max(dyslexia_score, 0.0)

def dyslexia_analysis(user_text, random_sentence):
    if is_exact_match(user_text, random_sentence):
        return 0.0

    words = tokenize_and_clean_text(user_text)
    if not words:
        return 0.0

    dyslexia_scores = []
    for word in words:
        dyslexia_scores.append(calculate_word_dyslexia_score(word, random_sentence))

    avg_score = round(sum(dyslexia_scores) / len(dyslexia_scores), 3)
    return avg_score

# ---- Questionnaire-based autism & ADHD scoring ----
# We will ask a short list of behavioral questions (you can edit these)
AUTISM_QUESTIONS = [
    ("respNamesBy9months", "Responds to name by 9 months? (never/lessOften/sometimes/moreOften): "),
    ("noPlaywithOtherby36Months", "Plays with other children by 36 months? (never/lessOften/sometimes/moreOften): "),
    ("noGestures12Months", "Used gestures by 12 months? (never/lessOften/sometimes/moreOften): "),
    ("repeatWords", "Repeats words or phrases (echolalia)? (never/lessOften/sometimes/moreOften): "),
    ("noAttention", "Has poor attention or focus? (never/lessOften/sometimes/moreOften): "),
    ("unusualEating", "Shows unusual eating behaviors? (never/lessOften/sometimes/moreOften): ")
]

ADHD_QUESTIONS = [
    ("hyperactivity", "Shows hyperactive behavior? (never/lessOften/sometimes/moreOften): "),
    ("forgetfulness", "Is forgetful in daily activities? (never/lessOften/sometimes/moreOften): "),
    ("disobeyInst", "Often disobeys instructions? (never/lessOften/sometimes/moreOften): "),
    ("repTempTantrum", "Frequent temper tantrums? (never/lessOften/sometimes/moreOften): "),
    ("noAttention", "Difficulty sustaining attention? (never/lessOften/sometimes/moreOften): "),
    ("harmingNature", "Shows impulsive harming behaviors? (never/lessOften/sometimes/moreOften): ")
]

def ask_question(prompt):
    while True:
        ans = input(prompt).strip()
        if ans in ANSWER_SCORE_MAP:
            return ans
        print("Invalid answer. Use one of: never, lessOften, sometimes, moreOften")

def autism_analysis_from_answers(answers_dict):
    # Score autism using weights: early-develop symptoms and social-communication get higher weight
    score = 0.0
    weights = {
        "respNamesBy9months": 2.5,
        "noPlaywithOtherby36Months": 2.5,
        "noGestures12Months": 2.5,
        "repeatWords": 1.8,
        "noAttention": 1.0,
        "unusualEating": 0.8
    }
    for key, weight in weights.items():
        val = ANSWER_SCORE_MAP.get(answers_dict.get(key, "never"), 0)
        score += val * weight

    # Normalize roughly to a 0-? range and set threshold
    # Higher score -> higher likelihood of autism
    return round(score, 3)

def adhd_analysis_from_answers(answers_dict):
    # ADHD scoring focuses on hyperactivity/impulsivity and inattention
    score = 0.0
    weights = {
        "hyperactivity": 2.2,
        "forgetfulness": 1.6,
        "disobeyInst": 1.0,
        "repTempTantrum": 0.6,
        "noAttention": 2.2,
        "harmingNature": 0.8
    }
    for key, weight in weights.items():
        val = ANSWER_SCORE_MAP.get(answers_dict.get(key, "never"), 0)
        score += val * weight

    return round(score, 3)

# Heuristic thresholds (transparent & adjustable)
AUTISM_THRESHOLD = 18.0   # above this -> "high likelihood"
ADHD_THRESHOLD = 12.0     # above this -> "high likelihood"

In [ ]:
# ---- PDF report generator (combined) ----
def generate_pdf_report(user_responses, dyslexia_scores, avg_score, dyslexia_verdict,
                        autism_score, autism_verdict,
                        adhd_score, adhd_verdict,
                        dyscalculia_score, dyscalculia_verdict, # Added dyscalculia parameters
                        user_name, user_id):
    pdf_filename = f"{user_name}_{user_id}_neurodev_report.pdf"
    c = canvas.Canvas(pdf_filename, pagesize=letter)
    width, height = letter

    # Title
    c.setFont("Helvetica-Bold", 16)
    title_text = "Neurodevelopmental Screening Report"
    title_x = (width - c.stringWidth(title_text, "Helvetica-Bold", 16)) / 2
    c.drawString(title_x, height - 40, title_text)

    y = height - 70
    c.setFont("Helvetica-Bold", 12)
    c.drawString(100, y, "Full Legal Name:")
    c.setFont("Helvetica", 12)
    c.drawString(220, y, user_name)

    y -= 18
    c.setFont("Helvetica-Bold", 12)
    c.drawString(100, y, "State ID Number:")
    c.setFont("Helvetica", 12)
    c.drawString(220, y, user_id)

    # Dyslexia section
    y -= 28
    c.setFont("Helvetica-Bold", 13)
    c.drawString(100, y, "Dyslexia Test")
    y -= 16
    c.setFont("Helvetica", 12)
    c.drawString(100, y, f"Average Dyslexia Score: {avg_score:.3f}")
    y -= 14
    c.drawString(100, y, f"Verdict: {dyslexia_verdict}")

    # list responses & scores
    y -= 22
    c.setFont("Helvetica-Bold", 13)
    c.drawString(100, y, "User Entered Sentences and Scores:")
    y -= 14
    c.setFont("Helvetica", 11)
    if user_responses:
        for i, (resp, score) in enumerate(zip(user_responses, dyslexia_scores), 1):
            if y < 80:
                c.showPage()
                y = height - 40
            c.drawString(110, y, f"{resp[:80]}")  # truncated display
            y -= 12
            c.drawString(130, y, f"Score: {score:.3f}")
            y -= 14
    else:
        c.drawString(110, y, "No sentences provided.")
        y -= 14

    # Autism section
    if y < 160:
        c.showPage()
        y = height - 40
    y -= 8
    c.setFont("Helvetica-Bold", 13)
    c.drawString(100, y, "Autism Screening")
    y -= 16
    c.setFont("Helvetica", 12)
    c.drawString(100, y, f"Autism Score: {autism_score:.3f}")
    y -= 14
    c.drawString(100, y, f"Verdict: {autism_verdict}")

    # ADHD section
    y -= 22
    c.setFont("Helvetica-Bold", 13)
    c.drawString(100, y, "ADHD Screening")
    y -= 16
    c.setFont("Helvetica", 12)
    c.drawString(100, y, f"ADHD Score: {adhd_score:.3f}")
    y -= 14
    c.drawString(100, y, f"Verdict: {adhd_verdict}")

    # Dyscalculia section (New)
    y -= 22
    c.setFont("Helvetica-Bold", 13)
    c.drawString(100, y, "Dyscalculia Screening")
    y -= 16
    c.setFont("Helvetica", 12)
    c.drawString(100, y, f"Dyscalculia Score: {dyscalculia_score:.3f}")
    y -= 14
    c.drawString(100, y, f"Verdict: {dyscalculia_verdict}")


    # Footer / signature
    y -= 40
    if y < 80:
        c.showPage()
        y = height - 40
    c.setFont("Helvetica-Bold", 12)
    c.drawString(100, y, "Clinician Signature: ________________________")
    c.save()
    print(f"PDF report '{pdf_filename}' generated successfully.")

# ---- Read sentences file helper ----
def read_sentences_from_file(filename):
    try:
        with open(filename, "r", encoding="utf-8") as file:
            sentences = [s.strip() for s in file.readlines() if s.strip()]
        return sentences
    except FileNotFoundError:
        print(f"File '{filename}' not found. Create a file named sentences.txt with sentences (one per line).")
        return []
    except Exception as e:
        print(f"An error occurred while reading '{filename}': {e}")
        return []

# ---- Main program ----
def main():
    try:
        user_responses = []
        dyslexia_scores = []

        user_name = input("\nPlease enter the patient's name: ").strip() or "anonymous"
        user_id = input("Please enter the patient's ID: ").strip() or "0000"

        # ---------- Dyslexia test (3 attempts) ----------
        print("\n=== Dyslexia sentence-copy test ===")
        max_attempts = 3
        sentences = read_sentences_from_file("sentences.txt")
        if not sentences:
            return

        for attempt in range(max_attempts):
            random_sentence = random.choice(sentences)
            print(f"\nGenerated Sentence (Attempt {attempt + 1}): {random_sentence}")
            while True:
                user_text = input("Enter the sentence for analysis: \n").strip()
                if not user_text:
                    print("Empty input. Try again.")
                    continue
                if is_exact_match(user_text, random_sentence) or are_strings_similar(user_text, random_sentence, SIMILARITY_THRESHOLD):
                    user_responses.append(user_text)
                    score = dyslexia_analysis(user_text, random_sentence)
                    dyslexia_scores.append(score)
                    print(f"Recorded. (score {score:.3f})")
                    break
                else:
                    print("Response is too different from the generated sentence. Please try again.")
                    time.sleep(0.5)
                    print(f"Copy this sentence: {random_sentence}")

        avg_dys_score = round(sum(dyslexia_scores) / len(dyslexia_scores), 3) if dyslexia_scores else 0.0
        dyslexia_verdict = "Likelihood of dyslexia detected." if avg_dys_score >= DYSLEXIA_SCORE_THRESHOLD else "No significant signs of dyslexia detected."
        print(f"\nAverage Dyslexia Score: {avg_dys_score:.3f} -> {dyslexia_verdict}")

        # ---------- Behavioral questionnaire ----------
        print("\n=== Short behavioral questionnaire (type one of: never, lessOften, sometimes, moreOften) ===")
        answers = {}

        print("\n-- Autism-related questions --")
        for key, prompt in AUTISM_QUESTIONS:
            answer = ask_question(prompt)
            answers[key] = answer

        print("\n-- ADHD-related questions --")
        for key, prompt in ADHD_QUESTIONS:
            # reuse answers dict (some items overlap)
            if key in answers:
                # don't repeat; just show existing value
                print(f"{prompt} (answered above as '{answers[key]}')")
            else:
                answer = ask_question(prompt)
                answers[key] = answer

        autism_score = autism_analysis_from_answers(answers)
        adhd_score = adhd_analysis_from_answers(answers)

        autism_verdict = "High likelihood of autism-related traits." if autism_score >= AUTISM_THRESHOLD else "Low / no significant autism-related traits detected."
        adhd_verdict = "High likelihood of ADHD-related traits." if adhd_score >= ADHD_THRESHOLD else "Low / no significant ADHD-related traits detected."

        print(f"\nAutism Score: {autism_score:.3f} -> {autism_verdict}")
        print(f"ADHD Score: {adhd_score:.3f} -> {adhd_verdict}")

        # ---------- Generate PDF ----------
        generate_pdf_report(user_responses, dyslexia_scores, avg_dys_score, dyslexia_verdict,
                            autism_score, autism_verdict, adhd_score, adhd_verdict,
                            user_name, user_id)

    except KeyboardInterrupt:
        print("\nOperation aborted by the user.")



In [ ]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True